In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
%cd ../

/workspace


# Import Module

In [53]:
import json
import os

import pandas as pd
import torch
from pyserini.search.lucene import LuceneSearcher
from tqdm.auto import tqdm

# Load Data

In [14]:
train_data_path = "./data/train.json"
test_data_path = "./data/test_data.json"
with open(train_data_path, "r") as f1, open(test_data_path, "r") as f2:
    train_data = json.load(f1)
    test_data = json.load(f2)
test_question = pd.read_csv("./data/test_questions.csv", encoding="utf8")
sample = pd.read_csv("./data/sample_submission.csv", encoding="utf8")
print(f"# of train data: {len(train_data['data']):,}")
print(f"# of test data: {len(test_data['data']):,}")

# of train data: 1,862
# of test data: 827


---

# BM25 Baseline

In [33]:
contexts = []

for data in test_data["data"]:
    for paragraph in data["paragraphs"]:
        contexts.append(
            {"id": paragraph["paragraph_id"], "contents": paragraph["context"]}
        )

with open("./data/index/test/doc.json", "w", encoding="utf8") as f:
    json.dump(contexts, f)

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input data/index/test/ \
  --language ko \
  --index data/test.index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 16 \
  --storePositions --storeDocvectors --storeRaw

In [35]:
searcher = LuceneSearcher("./data/test.index")
searcher.set_language("ko")

In [54]:
answer = []
for q in tqdm(test_question["question_text"]):
    answer.append(",".join([r.docid for r in searcher.search(q, k=10)]))
sample["paragraph_id"] = answer
sample.to_csv("submission.csv", index=False)

  0%|          | 0/11434 [00:00<?, ?it/s]